## 函数定义 + 读入

In [1]:
import os
import cv2
import numpy as np

# imshow wrapper
def snap(title, img):
    cv2.namedWindow(title, cv2.WINDOW_FREERATIO)
    cv2.imshow(title, img)
    # cv2.imwrite('res_dir/' + title + '.jpg', img)
    
    # 创建保存路径
    save_dir = "res_dir"
    os.makedirs(save_dir, exist_ok=True)
    
    # 保存图片到指定路径
    save_path = os.path.join(save_dir, title + '.jpg')
    cv2.imwrite(save_path, img)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 拉普拉斯金字塔图像融合
def laplacian_pyramid_blend(img1, img2, levels=6):
    # 生成高斯金字塔图像
    gaussian_pyr1 = [img1.astype(np.float32)]
    gaussian_pyr2 = [img2.astype(np.float32)]
    for _ in range(levels - 1):
        img1 = cv2.pyrDown(img1)
        img2 = cv2.pyrDown(img2)
        gaussian_pyr1.append(img1.astype(np.float32))
        gaussian_pyr2.append(img2.astype(np.float32))
    
    # 生成拉普拉斯金字塔图像
    laplacian_pyr1 = [gaussian_pyr1[levels - 1].astype(np.float32)]
    laplacian_pyr2 = [gaussian_pyr2[levels - 1].astype(np.float32)]
    for i in range(levels - 2, -1, -1):
        img1 = cv2.pyrUp(gaussian_pyr1[i + 1])
        img2 = cv2.pyrUp(gaussian_pyr2[i + 1])
        laplacian1 = cv2.subtract(gaussian_pyr1[i], img1)
        laplacian2 = cv2.subtract(gaussian_pyr2[i], img2)
        laplacian_pyr1.append(laplacian1)
        laplacian_pyr2.append(laplacian2)
    
    # 图像融合
    blended_pyr = []
    for lap1, lap2 in zip(laplacian_pyr1, laplacian_pyr2):
        rows, cols, _ = lap1.shape
        blended = np.hstack((lap1[:, :cols // 2], lap2[:, cols // 2:]))
        blended_pyr.append(blended)
    
    # 重建图像
    blended_img = blended_pyr[0]
    for i in range(1, levels):
        blended_img = cv2.pyrUp(blended_img)
        blended_img = cv2.add(blended_img, blended_pyr[i])
    
    return blended_img.astype(np.uint8)


# 裁剪黑边
def remove_black_borders(image):
    img = cv2.medianBlur(image, 5) #中值滤波，去除黑色边际中可能含有的噪声干扰
    b = cv2.threshold(img, 3, 255, cv2.THRESH_BINARY) #调整裁剪效果
    binary_image = b[1]
    binary_image = cv2.cvtColor(binary_image,cv2.COLOR_BGR2GRAY)
 
    edges_y, edges_x = np.where(binary_image==255) ##h, w
    bottom = min(edges_y)             
    top = max(edges_y) 
    height = top - bottom            
                                   
    left = min(edges_x)           
    right = max(edges_x)             
    height = top - bottom 
    width = right - left

    res_image = image[bottom:bottom+height, left:left+width]
    
    return res_image 
    

# 读取图像

prefix = 'image/'
sort = 'a'

image1 = cv2.imread(prefix + sort + '1.jpg')
image2 = cv2.imread(prefix + sort + '2.jpg')

image1 = cv2.resize(image1, (0, 0), fx=0.5, fy=0.5)  # 调整比例为0.5，可以根据需要调整比例
image2 = cv2.resize(image2, (0, 0), fx=0.5, fy=0.5)


## 特征提取 + 匹配

In [2]:
sift = cv2.SIFT.create()

keyPoint1, descriptor1 = sift.detectAndCompute(image1, None)
keyPoint2, descriptor2 = sift.detectAndCompute(image2, None)

# BF
# bf = cv2.BFMatcher(cv2.NORM_L2)  # sift的normType应该使用NORM_L2或者NORM_L1
# rawMatches = bf.knnMatch(descriptor1, descriptor2,2)# drawMatchesKnn

# 创建FLANN匹配器
indexParams = dict(algorithm = 0, trees = 5)
searchParams = dict(checks=50)
flann=cv2.FlannBasedMatcher(indexParams,searchParams)
rawMatches=flann.knnMatch(descriptor1,descriptor2, k=2)


matches = []
for m in rawMatches:
    # 当最近距离跟次近距离的比值小于ratio值时，保留此匹配对
    if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
    # 存储两个点在featuresA, featuresB中的索引值
        matches.append(m[0])


if len(matches) <= 8:
    print("fail")
    exit()

## 拼接

In [3]:
# 显示匹配
match_img = cv2.drawMatches(image1, keyPoint1, image2, keyPoint2, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
snap('01_matches', match_img)


# 获取匹配对的点坐标
pts1 = np.float32([keyPoint1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
pts2 = np.float32([keyPoint2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
# 计算视角变换矩阵
H,mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)

    
# 执行变换
# warp = cv2.warpPerspective(image2, np.linalg.inv(H), (image1.shape[1] + image2.shape[1], image2.shape[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 255, 0))
warp = cv2.warpPerspective(image2, np.linalg.inv(H), (image1.shape[1] + image2.shape[1], image1.shape[0] + image2.shape[0]))


# 右图变换后
snap('02_warp', warp)


# 未融合图像
res_raw = warp.copy();
res_raw[0:image1.shape[0], 0:image2.shape[1]] = image1
snap('03_res_raw', res_raw)


## 融合

In [4]:
# 获取蒙版
mask_w = np.ones_like(image2, dtype=np.uint8) * 255
mask_r = cv2.warpPerspective(mask_w, np.linalg.inv(H), (image1.shape[1] + image2.shape[1], image1.shape[0] + image2.shape[0]))
mask = (mask_r == 255).all(axis=2)
    
# 构造融合
asm1 = warp.copy()
asm1[0:image1.shape[0], 0:image2.shape[1]] = image1

asm2 = warp.copy()
asm2.fill(0);
asm2[0:image1.shape[0], 0:image2.shape[1]] = image1
    
tmp = warp.copy()
asm2[mask] = tmp[mask] # 使用蒙版将图像A的对应像素复制到图像B中

snap('04_asm1', asm1)
snap('05_asm2', asm2)


# 融合图像
# result = cv2.addWeighted(asm1, 0.5, asm2, 0.5, 0)
result = laplacian_pyramid_blend(asm1, asm2)
snap('06_res_blend', result)


#裁剪黑边
result = remove_black_borders(result)


# 结果
snap('07_result', result)


print("success")
exit()




success
